In [32]:
### en este archivo se genera el archivo
#-Institutions que hace referencia a "static_institutions"
#-affiliations que hace referencia a "static_affiliations"
#-affiliations para notebook testeo

import pickle
#!pip install boto3
import boto3
#!pip install fsspec
import fsspec
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#!pip install pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = SparkContext._active_spark_context #devuelve la instancia existente
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, FloatType, ArrayType, DoubleType, StructType, StructField
sqlContext = SQLContext(sc,spark)

#from google.colab import drive

import os

#!pip install pyalex
import pyalex
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
from pyalex import config

In [33]:
#drive.mount('/content/drive') ### comentar en script

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
base_path = '../../../'

In [37]:
config.max_retries = 0
config.retry_backoff_factor = 0.1
config.retry_http_codes = [429, 500, 503]

pager = Institutions().filter(country_code="AR").paginate(method="page",per_page=200)

listaDeInstituciones = list()
for page in pager:
    print(len(page))
    listaDeInstituciones += page

print(len(listaDeInstituciones))

institutions_df=pd.DataFrame(listaDeInstituciones)
institutions_df['affiliation_id'] = institutions_df['id'].apply(lambda x: x.split("/")[-1])
institutions_df['affiliation_id']=institutions_df['affiliation_id'].apply(lambda x: x.split("I")[-1])
institutions_df['ror_id'] = institutions_df['ror'].apply(lambda x: x.split("/")[-1])
print(institutions_df.head(n=2))
print(institutions_df.shape)

institutions_df.to_csv(f'{base_path}Crudos/institutions.csv',index=False)
institutions_df.to_parquet(f'{base_path}Crudos/OA_static_institutions_single_file.parquet')

200
131
0
331
                                id                        ror  \
0  https://openalex.org/I151201029  https://ror.org/03cqe8w59   
1   https://openalex.org/I24354313  https://ror.org/0081fs513   

                                        display_name country_code        type  \
0  Consejo Nacional de Investigaciones Científica...           AR  government   
1                         University of Buenos Aires           AR   education   

                                             type_id  \
0  https://openalex.org/institution-types/government   
1   https://openalex.org/institution-types/education   

                                             lineage  \
0  [https://openalex.org/I151201029, https://open...   
1                   [https://openalex.org/I24354313]   

                homepage_url  \
0  http://www.conicet.gov.ar   
1         http://www.uba.ar/   

                                           image_url  \
0  https://commons.wikimedia.org/w/index.php?titl...   

In [38]:
#######################################################
#### Generación archivo de affiliations a partir de authorship
#######################################################
pager = Works().filter(authorships={"institutions": {"country_code": "AR"}}).paginate(method="page",per_page=200)

listaDeWorks = list()
for page in pager:
    print(len(page))
    listaDeWorks += page

argentina_works=pd.DataFrame(listaDeWorks)
argentina_works['affiliations'] = argentina_works['authorships'].apply(lambda x: [i['affiliations'] for i in x] if isinstance(x,list) else [])
argentina_works['raw_affiliation_string'] = argentina_works['affiliations'].apply(lambda x: [d.get('raw_affiliation_string') for sublist in x for d in sublist if isinstance(sublist, list) and isinstance(d, dict)] if isinstance(x, list) else [])
argentina_works['institution_ids']        = argentina_works['affiliations'].apply(lambda x: [d.get('institution_ids') for sublist in x for d in sublist if isinstance(sublist, list) and isinstance(d, dict)] if isinstance(x, list) else [])
argentina_works=argentina_works.explode('institution_ids')
argentina_works=argentina_works[['id','institution_ids','raw_affiliation_string']]
print(argentina_works.head(2))

# Ensure that 'institution_ids' and 'raw_affiliation_string' have the same number of elements for each row.
argentina_works['zipped_data'] = argentina_works.apply(
    lambda row: list(zip(row['institution_ids'], row['raw_affiliation_string']))
    if len(row['institution_ids']) == len(row['raw_affiliation_string']) else [], axis=1
)
# Filter out rows with empty zipped_data (unequal length entries)
affiliations = argentina_works[argentina_works['zipped_data'].apply(len) > 0]

# Explode the zipped column
affiliations = affiliations.explode('zipped_data')

# Split the zipped data back into original columns
affiliations[['institution_ids', 'raw_affiliation_string']] = pd.DataFrame(
    affiliations['zipped_data'].tolist(), index=affiliations.index
)
affiliations['works_id'] = affiliations['id'].apply(lambda x: x.split("/")[-1])
affiliations['institution_ids'] = affiliations['institution_ids'].apply(lambda x: x.split("/")[-1])
affiliations['institution_ids'] =affiliations['institution_ids'] .apply(lambda x: x.split("I")[-1])
affiliations['affiliation_id']=affiliations['institution_ids']
affiliations['original_affiliation']=affiliations['raw_affiliation_string']

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
                                 id                    institution_ids  \
0  https://openalex.org/W2746485780  [https://openalex.org/I149521062]   
0  https://openalex.org/W2746485780   [https://openalex.org/I12834331]   

                              raw_affiliation_string  
0  [Center for Environmental and Estuarine Studie...  
0  [Center for Environmental and Estuarine Studie...  


In [39]:
#######################################
##se arma archivo para notebook de test
#######################################
affiliation_muestra_para_testeo=affiliations.sample(frac=0.1)
affiliation_muestra_para_testeo=affiliation_muestra_para_testeo[['institution_ids','raw_affiliation_string','works_id']]
affiliation_muestra_para_testeo=affiliation_muestra_para_testeo.rename(columns={'institution_ids':'labels', 'works_id': 'paper_id', 'raw_affiliation_string': 'affiliation_string'})
affiliation_muestra_para_testeo.to_csv(f"{base_path}Crudos/affiliations_para_testeo.csv")

In [40]:
##A affiliations le restamos las observaciones que vamos a usar para testear
affiliations=affiliations-affiliation_muestra_para_testeo

In [41]:
#####################################
# Guardamos affiliations como parquet
#####################################
affiliations.to_csv(f"{base_path}Crudos/affiliations.csv")
affiliations.to_parquet(f"{base_path}Crudos/static_affiliations.parquet")